In [1]:
import torch
import pandas as pd
import numpy as np
import pickle
import torchvision.models as models

In [21]:
def exShapeMatrix(m_path='ShapeNet.pth'):
    pretrained_dict = torch.load(m_path)
    M = pretrained_dict['model']['classifier.0.weight']
    # resave
    torch.save({'shape2vec': M}, "shape2vec")
    print("Saved!")
# M = pretrained_dict['model']['classifier.0.weight'].cpu().detach().numpy()

exShapeMatrix()

def get_label_dict():
    f = open('./chinese_labels', 'rb')
    label_dict = pickle.load(f)
    f.close()
    return label_dict

id2char = get_label_dict()
char2id = {x: y for x, y in zip(id2char.values(), id2char.keys())}

Saved!


In [25]:
def getAllReady():
    id2char = get_label_dict()
    char2id = {x: y for x, y in zip(id2char.values(), id2char.keys())}
    M = torch.load('./shape2vec')['shape2vec']
    print(M.shape)
    return M
    
M = getAllReady()

torch.Size([3755, 300])


In [26]:
def getTopKSim(q_char, K=20):
    id = char2id[q_char]
    sim_lst = []
    for i in range(M.shape[0]):
        sim = torch.cosine_similarity(M[id], M[i], dim=-1)
        sim_lst.append((sim, i))
    sim_lst.sort(key=lambda x: x[0], reverse=True)
    topK_id = sim_lst[1:K+1]
    topK_char = []
    for _, id in topK_id:
        topK_char.append(id2char[id])
    return topK_char


In [27]:
print(getTopKSim("皿", 20))

['血', '肌', '亚', '盟', '显', '业', '圆', '僵', '见', '风', '吧', '哑', '凰', '咀', '趾', '瓜', '圃', '遭', '迎', '狙']


In [ ]:
def getTopKSimDict(K=20):
    sim_dct = {}
    for q_char in char2id.keys():
        id = char2id[q_char]
        sim_lst = []
        for i in range(M.shape[0]):
            sim = torch.cosine_similarity(M[id], M[i], dim=-1)
            sim_lst.append((sim, i))
        sim_lst.sort(key=lambda x: x[0], reverse=True)
        topK_id = sim_lst[1:K+1]
        topK_char = []
        for _, id in topK_id:
            topK_char.append(id2char[id])
        sim_dct[q_char] = topK_char
    return sim_dct

sim_dct = getTopKSimDict()
torch.save(sim_dct, "./sim_dct")